# Giảm Lịch Sử Trò Chuyện với Agent Scratchpad trong Semantic Kernel

Notebook này minh họa cách sử dụng tính năng Giảm Lịch Sử Trò Chuyện của Semantic Kernel cùng với một agent scratchpad để duy trì ngữ cảnh trong các cuộc trò chuyện. Đây là yếu tố quan trọng để xây dựng các tác nhân AI hiệu quả có thể xử lý các cuộc trò chuyện dài mà không vượt quá giới hạn token.

## Bạn Sẽ Học Được Gì:
1. **Giảm Lịch Sử Trò Chuyện**: Cách tự động tóm tắt lịch sử cuộc trò chuyện để quản lý việc sử dụng token
2. **Agent Scratchpad**: Một hệ thống bộ nhớ liên tục để theo dõi sở thích của người dùng và các nhiệm vụ đã hoàn thành
3. **Theo Dõi Sử Dụng Token**: Giám sát cách sử dụng token thay đổi khi có và không có giảm lịch sử

## Yêu Cầu Trước:
- Cài đặt Azure OpenAI với các biến môi trường đã được cấu hình
- Hiểu các khái niệm cơ bản về agent từ các bài học trước


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## Hiểu về Agent Scratchpad

### Agent Scratchpad là gì?

**Agent Scratchpad** là một hệ thống bộ nhớ liên tục mà các agent sử dụng để:
- **Theo dõi các nhiệm vụ đã hoàn thành**: Ghi lại những gì đã được thực hiện cho người dùng
- **Lưu trữ sở thích của người dùng**: Nhớ các điều thích, không thích và yêu cầu
- **Duy trì ngữ cảnh**: Giữ thông tin quan trọng để sử dụng trong các cuộc trò chuyện
- **Giảm sự lặp lại**: Tránh hỏi lại những câu hỏi giống nhau nhiều lần

### Cách hoạt động:
1. **Hoạt động ghi**: Agent cập nhật scratchpad sau khi học được thông tin mới
2. **Hoạt động đọc**: Agent tham khảo scratchpad khi đưa ra quyết định
3. **Tính bền vững**: Thông tin vẫn được lưu giữ ngay cả khi lịch sử trò chuyện bị rút gọn

Hãy nghĩ về nó như cuốn sổ tay cá nhân của agent, bổ sung cho lịch sử cuộc trò chuyện.


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## Tạo Plugin Agent Scratchpad

Plugin này cho phép agent đọc và ghi vào một tệp scratchpad lưu trữ.


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## Khởi tạo Bộ giảm lịch sử trò chuyện

ChatHistorySummarizationReducer tự động tóm tắt lịch sử cuộc trò chuyện khi vượt quá ngưỡng.


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## Tạo Đại lý Lập kế hoạch Kỳ nghỉ

Đại lý này sẽ giúp người dùng lập kế hoạch kỳ nghỉ trong khi duy trì ngữ cảnh thông qua scratchpad.


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## Thực hiện Cuộc trò chuyện Lập kế hoạch Kỳ nghỉ

Bây giờ hãy cùng thực hiện một cuộc trò chuyện hoàn chỉnh để minh họa:
1. Yêu cầu lập kế hoạch ban đầu
2. Thu thập sở thích
3. Tạo lịch trình
4. Thay đổi địa điểm
5. Giảm lịch sử trò chuyện
6. Sử dụng bảng ghi chú


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## Phân tích kết quả

Hãy cùng phân tích những gì đã xảy ra trong cuộc trò chuyện của chúng ta:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## Những Điểm Chính

### 1. Giảm Lịch Sử Trò Chuyện
- **Kích Hoạt Tự Động**: Giảm lịch sử xảy ra khi số lượng tin nhắn vượt ngưỡng cho phép
- **Tiết Kiệm Token**: Giảm đáng kể số lượng token sau khi tóm tắt
- **Bảo Lưu Ngữ Cảnh**: Thông tin quan trọng được giữ lại trong các bản tóm tắt

### 2. Lợi Ích Của Sổ Tay Tạm Thời Của Agent
- **Bộ Nhớ Bền Vững**: Sở thích của người dùng vẫn được lưu giữ sau khi giảm lịch sử
- **Theo Dõi Nhiệm Vụ**: Agent duy trì ghi chép về công việc đã hoàn thành
- **Trải Nghiệm Cải Thiện**: Không cần lặp lại các sở thích

### 3. Mô Hình Sử Dụng Token
- **Tăng Trưởng Tuyến Tính**: Token tăng lên theo từng tin nhắn
- **Giảm Đột Ngột**: Việc giảm lịch sử làm giảm đáng kể số lượng token
- **Cuộc Trò Chuyện Bền Vững**: Cho phép tương tác lâu hơn trong giới hạn


## Dọn dẹp

Dọn dẹp tệp scratchpad được tạo trong suốt buổi trình diễn này:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# Tóm tắt

Chúc mừng! Bạn đã triển khai thành công một tác nhân AI với khả năng quản lý ngữ cảnh tiên tiến:

## Những điều bạn đã học:
- **Giảm lịch sử trò chuyện**: Tự động tóm tắt các cuộc hội thoại để quản lý giới hạn token
- **Bảng ghi chú của tác nhân**: Triển khai bộ nhớ liên tục cho các sở thích của người dùng và các nhiệm vụ đã hoàn thành
- **Quản lý token**: Theo dõi và tối ưu hóa việc sử dụng token trong các cuộc trò chuyện dài
- **Bảo toàn ngữ cảnh**: Duy trì thông tin quan trọng qua các lần giảm cuộc hội thoại

## Ứng dụng thực tế:
- **Bot dịch vụ khách hàng**: Ghi nhớ sở thích của khách hàng qua các phiên làm việc
- **Trợ lý cá nhân**: Theo dõi các dự án đang thực hiện và thói quen của người dùng
- **Gia sư giáo dục**: Duy trì tiến độ học tập và sở thích học tập của học sinh
- **Trợ lý chăm sóc sức khỏe**: Lưu giữ lịch sử bệnh nhân trong khi vẫn tôn trọng giới hạn token

## Bước tiếp theo:
- Triển khai các mô hình bảng ghi chú phức tạp hơn
- Thêm lưu trữ cơ sở dữ liệu cho các tình huống nhiều người dùng
- Tạo chiến lược giảm ngữ cảnh tùy chỉnh cho các nhu cầu cụ thể của từng lĩnh vực
- Kết hợp với cơ sở dữ liệu vector để tìm kiếm bộ nhớ ngữ nghĩa
- Xây dựng các tác nhân có thể tiếp tục cuộc trò chuyện sau nhiều ngày với đầy đủ ngữ cảnh



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
